In [1]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interactive, widgets, interact
from IPython.core.display import HTML, display
import plotly.graph_objects as go
import json
import pickle
from hdfs3 import HDFileSystem
from wybe import *

In [2]:
# Load Important Data for visualization

# Graph for computing routing
G,stop_name_to_id = Utils.read_graph_from_hdfs("2")

# Df with mapping from Trip id to route desc and route short name
trip_route_name = Utils.load_hdfs_to_pandas('trips_route_name.parquet')

# Translation table from trip id to line id to show to user
dict_trip_route_name = trip_route_name.set_index('trip_id')['route_short_name'].to_dict()

# Loads all stop names within 15km
hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille')
with hdfs.open('/user/boesinge/finalproject/stops_names.pkl', 'rb') as f:
    b = f.read()
    stop_names = list(pickle.loads(b))
    
# Creating the routing class for later
routing = Routing(G, stop_name_to_id)

In [3]:
### The next cell runs the voilà notebook so you don't have to !

In [3]:
%%javascript

let s = window.location.href
if(s.includes('/lab')){
window.open(s.split('/lab')[0] + '/voila/render/notebooks/vis.ipynb')
}


<IPython.core.display.Javascript object>

In [7]:
# Method to squeeze the dataframes to have the edges of the same trip represented in a single entry
def squeeze_df(df):
        # Get Intermediate stop for each trip in route dataframe
        intermediate_stops = df[['trip_id','dep_stop_name']].groupby('trip_id')\
                .agg({'dep_stop_name':lambda x : list(x)[1:]})\
                .reset_index().rename(columns={'dep_stop_name':'intermediate_stops'})
        
        # Select the values to be kept for each trip
        df = df.groupby('trip_id')\
                .agg({'ttype':'first','dep_stop_id':'first','arr_stop_id':'last','dep_time':'first','arr_time':'last',
                     'dep_stop_name':'first','arr_stop_name':'last','dep_lat':'first','dep_lon':'first','arr_lat':'last',
                      'arr_lon':'last','travel_time':'sum'})\
                .reset_index()
        # Merge stops with information about trip and return resulting dataframe
        return df.merge(intermediate_stops,on='trip_id').sort_values(by='dep_time')

In [8]:
def pretty_df(df):
    # Rename the different columns so that the information to be displayed is a more readable format
    df = (df.reset_index()[['ttype','trip_id','dep_time','arr_time','dep_stop_name','arr_stop_name','travel_time','intermediate_stops']]
          .rename(columns= {"trip_id": "Line","ttype":"Transport Type","dep_stop_name":"Departure Stop","arr_stop_name":"Arrival Stop",
                            "dep_time":"Departure Time","arr_time":"Arrival Time",'travel_time':"Travel Time",'intermediate_stops':"Intermediate Stops"})
         )
    
    # Reformat Intermediate stops to handle the case of direct print and improve reading format
    df['Intermediate Stops'] = df['Intermediate Stops'].apply(lambda x : "Direct Trip" if (len(x)==0) else '|'.join(x) if len(x) <= 2 else '|'.join(x[:2]) + '..')
    
    # Transform unix time into timestamp for readability
    df["Departure Time"] = df["Departure Time"].apply(lambda x : Utils.print_timestamp(x))
    df["Arrival Time"] = df["Arrival Time"].apply(lambda x : Utils.print_timestamp(x))
    df["Travel Time"] = df["Travel Time"].apply(lambda x : Utils.print_timestamp(x))
    
    # Transform trip id in line name eg. Tram 9, Bus 134
    df["Line"] = df['Transport Type'] + ' ' + df['Line'].apply(lambda x : dict_trip_route_name.get(x, ''))
    return df

In [9]:
# Method to create a styler dataframe to have a nicer visualization of the dataframe
def pretty_style(df):
    # Colors for each type of transport
    coolors = {
    'Bus': '#FEC89A',
    'Tram':'#ECE4DB',
    'Foot': '#F8EDEB'
    }
    # Return the styler with background color corresponding to transport type color
    return df.style.hide_columns(['Transport Type']).apply(lambda x: [f"background-color: {coolors.get(x['Transport Type'],'#FEC5BB')}" for y in x],axis=1)

In [10]:
# Method to call the planner with given arguments 
def compute_journey(start, stop, arrival, confidence):
    if not start or not stop:
        return
    route = routing.robust(start, stop, f'{arrival}:00',threshold = confidence, number_of_routes=3)
    return route

# Method to plot a journey in a map 
def plot_journey(route):
    
    # Dict of markers for each transport type
    markers = {
    'Bus': 'bus',
    'Foot':'toilet',
    'Init': 'circle'
    }
    
    
    # Transform route to dataframe
    dataframe = route.to_Pandas()
    
    # Get squeezed version of dataframe
    squeezed_df = squeeze_df(dataframe)
    
    # Dataframe with only the rows which are line changes
    first_take = dataframe.drop_duplicates(subset=['trip_id'])
    
    # Plot of the polyline between all stops of the path
    fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lat = list(dataframe.dep_lat.values)+[dataframe.iloc[-1].arr_lat],
    lon = list(dataframe.dep_lon.values)+[dataframe.iloc[-1].arr_lon],
    hovertext=list(dataframe.dep_stop_name)+[dataframe.iloc[-1].arr_stop_name],
    marker = {'size': 10, 'symbol':list(dataframe.apply(lambda x : markers.get(x.ttype,'rail') if (x.name in first_take.index.values) else 'square' ,axis=1))+['embassy']}))
    
    # Params for the map style
    fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox=dict(
        accesstoken="pk.eyJ1IjoibGdpb3JkYW4iLCJhIjoiY2tvMXlqaXczMHVjYzJvazRyNXoyZmpxeCJ9.rYkK9im_fzp_XShXcMmbdA",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=47.377220,
            lon=8.539902
        ),
        pitch=0,
        zoom=12
    ))
    fig.show()
    
    display(HTML(pretty_style(pretty_df(squeezed_df)).hide_index().render().replace('<table','<table style="width:100%"')))


# selector for start station
start_selector = widgets.Combobox(
    placeholder='Choose a starting point',
    options= stop_names,
    description='Departure:',
    ensure_option=True,
    disabled=False
)

# selector for end station
stop_selector = widgets.Combobox(
    placeholder='Choose an ending point',
    options= stop_names,
    description='Arrival:',
    ensure_option=True,
    disabled=False
)

# Time selector
times = [x.time().strftime('%H:%M') for x in pd.date_range("08:00", "22:00", freq="1min")]

# Arrival time selector
arrival = widgets.SelectionSlider(
    value='12:00',
    options=times,
    description='Arrival Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

# Confidence value selector
conf = widgets.FloatSlider(
    value=0.7,
    min=0.01,
    max=0.99,
    step=0.01,
    description='Confidence:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# Go Button to start computation
button = widgets.Button(description="Go")

# Add gif to display it while algorithm is running
file = open("train.gif", "rb")
image = file.read()
loading = widgets.Image(
    value=image,
    format='png',
    width=500,
)

# Layout params
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

img_box = widgets.HBox(children=[loading],layout=box_layout)


In [11]:
# Code for Header of our voila page
HTML("""
<div class="header">
  <img src="Connected.png" style="height:600px;margin-top: -120px;margin-bottom : -80px ">
</div>

<style>
.header {
  padding: 0px;
  text-align: center;
  background: #D8E2DC;
  color: white;
  font-size: 20px;
  overflow : hidden;
}

</style>
""")

In [12]:
# Params of the different paths to display to the users
colors = ['#4d4995', '#ea175a', '#fdc30a']
nums = ['First', 'Second', 'Third']
trip_header = """
<div class="header" style="padding:">
  <h1 style="color:{color}"> {num} Proposed Trip, with probability of success : {proba:2.3f}</h1>
</div>
""" 

In [13]:
# Add header in the page
box_layout = widgets.Layout(display='flex',justify_content='center')

# Add the selectors to run the algorithm 
ui = widgets.HBox([start_selector, stop_selector, arrival, conf,button], layout=box_layout)
ui.add_class('center')

out = widgets.Output()

# Method to launch computation of the algorithm and display resulting paths 
def showOutput(btn):
    # Start computation only if the two selectors with no default value are setted 
    if start_selector.value and stop_selector.value:
        with out:
            # Clear output after each computation 
            out.clear_output()
            display(img_box)
            
            # Compute route with the values of the selectors
            routes = compute_journey(start_selector.value, stop_selector.value, arrival.value, conf.value)
            
            # Squezz the dataframes with all the values to get only the changes in the path
            squezzed_dfs = [(x, x.to_Pandas().iloc[0].dep_time,len(squeeze_df(x.to_Pandas()))) for x in routes]
            
            # Sort routes first based on departure time and otherwise by number of changes
            routes = [x[0] for x in sorted(squezzed_dfs,key=lambda y : (-y[1],y[2]))]
            out.clear_output()
            
            # Display each path returned by our planner
            for i,route in enumerate(routes[:3]):
                display(HTML(trip_header.format(color = colors[i], num = nums[i],proba=route.success_proba()[0])))
                plot_journey(route)
# Once the button is clicked we launch the computation   
button.on_click(showOutput)
       
display(ui)
display(out)

Output()